In [1]:
import nbaPlayerStatModeling as nbaModel

In [2]:
nba = nbaModel.playerStatModel(
        day_offset = 0, 
        season = '2024-25', 
        perMode = 'PerGame',
        num_simulations = 200000
)

In [3]:
nba.get_teams_playing()
nba.get_teams_data(
    base_url = 'https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType={stattype}&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={sid}&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='
)
nba.get_players_playing(
    url_base_nba_player_stat = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode={per}&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={sid}&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID={tid}&TwoWay=0&VsConference=&VsDivision=&Weight=',
    minute_cutoff = 0
)
nba.get_props()

5  games today...
168 players returned from 10 teams, out of a total 10
745 prop bets for 76 players...


In [ ]:
#nba.df_players = nba.df_players.iloc[1:10, :]
nba.get_player_home_adv(
    use_default=False
)

In [65]:
#df_copy = nba.df_players.copy()
nba.df_players = df_copy.copy()

In [66]:
nba.model_expected_minutes()
nba.calculate_model_inputs()
nba.df_players = nba.df_players.fillna(0)
nba.num_simulations = 10000
nba.model_expected_stats_poisson()

In [67]:
nba.model_expected_pts()

In [68]:
nba.model_expected_fg3m()

In [69]:
nba.df_players[nba.df_players['FG3A']>4][['PLAYER_NAME', 'FG3M', 'FG3A' , 'expFG3M','threes_line', 'threes_oOdds', 'FG3MoOdds','FG3MoOdds_deci','FG3MoProb']]

,PLAYER_NAME,FG3M,FG3A,expFG3M,threes_line,threes_oOdds,FG3MoOdds,FG3MoOdds_deci,FG3MoProb
1,CJ McCollum,3.1,8,3.761090,3.5,118.0,189.0,2.891,0.3459
2,Jordan Hawkins,1.9,5,2.077916,1.5,-120.0,-113.0,1.879,0.5321
3,Jose Alvarado,2.1,5,2.595702,2.5,105.0,231.0,3.312,0.3019
4,Trey Murphy III,3.2,8,3.584768,3.5,105.0,178.0,2.781,0.3596
7,Klay Thompson,3.0,7,3.092198,3.5,106.0,264.0,3.643,0.2745
8,Kyrie Irving,2.9,7,2.316674,2.5,-166.0,-157.0,1.637,0.6110
13,Buddy Hield,2.6,7,1.979509,2.5,-130.0,-102.0,1.974,0.5065
19,Stephen Curry,4.4,11,4.471347,4.5,108.0,128.0,2.289,0.4369
23,Dillon Brooks,2.4,6,2.080324,2.5,-105.0,133.0,2.335,0.4283
25,Jalen Green,2.9,8,2.257529,3.5,124.0,254.0,3.549,0.2818


In [48]:
nba.df_players.columns#[['FG3A', 'FG3_PCT']].dtypes

Index(['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'MIN',
       'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       ...
       'REBoOdds_deci', 'ASToProb', 'ASToOdds', 'ASToOdds_deci', 'PTSoProb',
       'PTSoOdds', 'PTSoOdds_deci', 'FG3MoProb', 'FG3MoOdds',
       'FG3MoOdds_deci'],
      dtype='object', length=128)

In [30]:
player = 'Kenrich Williams'

In [31]:
nba.df_players[nba.df_players['PLAYER_NAME']==player][['PLAYER_NAME', 'REB', 'expReb', 'reb_line', 'reb_oOdds', 'REBoOdds','REBoOdds_deci','REBoProb']]

,PLAYER_NAME,REB,expReb,reb_line,reb_oOdds,REBoOdds,REBoOdds_deci,REBoProb
65,Kenrich Williams,3.4,3.11407,3.5,-100.0,164,2.646,0.377965


In [32]:
nba.df_players[nba.df_players['PLAYER_NAME']==player][['PLAYER_NAME', 'AST', 'expAst', 'ast_line', 'ast_oOdds', 'ASToOdds','ASToOdds_deci','ASToProb']]

,PLAYER_NAME,AST,expAst,ast_line,ast_oOdds,ASToOdds,ASToOdds_deci,ASToProb
65,Kenrich Williams,1.2,1.347933,1.5,115.0,157,2.57,0.389085


In [33]:
nba.df_players[nba.df_players['PLAYER_NAME']==player][['PLAYER_NAME', 'PTS', 'expPts', 'pts_line', 'pts_oOdds', 'PTSoOdds','PTSoOdds_deci','PTSoProb']]

,PLAYER_NAME,PTS,expPts,pts_line,pts_oOdds,PTSoOdds,PTSoOdds_deci,PTSoProb
65,Kenrich Williams,5.9,6.182814,5.5,105.0,-109,1.911,0.5232


In [60]:
nba.df_players.to_excel('test.xlsx')

In [49]:
nba.df_players['expAst'].values

array([5.12279194, 4.98906653, 2.06787421, 8.85399571, 4.02008345])

In [46]:
np.random.poisson(nba.df_players['expAst'].values[:, None], (len(nba.df_players), 10000))

array([[ 1,  7,  8, ...,  6,  7,  4],
       [ 9,  4,  2, ...,  6,  6,  3],
       [ 3,  1,  2, ...,  1,  4,  3],
       [14,  8,  6, ..., 12,  5,  9],
       [ 5,  5,  5, ...,  6,  7,  5]])

In [50]:
(len(nba.df_players), 10000)

(5, 10000)

In [105]:
for i in nba.df_players.columns:
    if 'reb' in i.lower():
        print(i)

OREB
DREB
REB
OREB_PCT
DREB_PCT
OREBadj
DREBadj
reb_line
reb_oOdds
reb_uOdds
OREB_HOMEadj
DREB_HOMEadj
REB_HOMEadj
OREB_ROADadj
DREB_ROADadj
REB_ROADadj
expReb
REBoProb
REBoOdds
REBoOdds_deci


In [ ]:
from nba_api.stats.endpoints import PlayerDashboardByGeneralSplits

############### DF IDX 6 is days rest splits
a.get_data_frames()[6]

In [26]:
a.get_data_frames()[6]

,GROUP_SET,GROUP_VALUE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK
0,Days Rest,0 Days Rest,9,4,5,0.444,303.761667,51,139,0.367,...,1,5,6,3,3,1,2,2,1,2
1,Days Rest,1 Days Rest,28,12,16,0.429,1033.046667,210,512,0.410,...,2,7,7,1,1,7,1,1,1,1
2,Days Rest,2 Days Rest,7,3,4,0.429,251.258333,51,118,0.432,...,2,6,5,2,2,4,3,3,1,3
3,Days Rest,3 Days Rest,2,2,0,1.000,70.165000,18,38,0.474,...,2,3,2,5,5,2,5,4,1,5
4,Days Rest,4 Days Rest,2,1,1,0.500,74.778333,20,47,0.426,...,6,4,4,4,4,5,4,4,1,4
5,Days Rest,5 Days Rest,1,1,0,1.000,33.125000,5,18,0.278,...,6,2,2,7,7,6,7,7,1,7
6,Days Rest,6+ Days Rest,1,1,0,1.000,36.678333,7,16,0.438,...,5,1,1,5,6,2,6,4,1,6
